In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import dual_annealing
from openpyxl import load_workbook

In [8]:
# Function to import the data from the Excel file
def load_datasets(filename='benchmark_target.xlsx'):
    df_benchmark = pd.read_excel(filename, sheet_name='Benchmark')
    df_target = pd.read_excel(filename, sheet_name='Target')
    return df_benchmark, df_target

In [9]:
# Function to calculate the necessary metrics for both datasets
def calculate_metrics(df, target_size, weights):
    clients_with_payments = df['LP'].notnull().sum() / len(df)  # % of clients with payments
    total_paid = df['Payments'].sum()  # Total paid
    total_debt = df['Deb'].sum()  # Total debt
    collection_percentage = total_paid / (total_debt + total_paid) if total_debt + total_paid != 0 else 0  # % of collection

    averages = df.mean(skipna=True)  # Calculate averages for all columns, skipping empty cells

    return {
        'Deb': averages['Deb'],
        'GR': averages['GR'],
        'DPD': averages['DPD'],
        'LP': averages['LP'],  # Average of LP column
        'collection': collection_percentage,  # % of collection
        'payments': clients_with_payments  # % of clients with payments
    }

In [10]:
# Objective function to minimize the difference between benchmark and target metrics
def objective_function(x, df_benchmark, df_target, target_metrics, weights):
    selected_indices = [int(i) for i in x]  # Convert row numbers to integers
    sample = df_benchmark.iloc[selected_indices]  # Select rows based on indices

    sample_metrics = calculate_metrics(sample, len(selected_indices), weights)  # Calculate metrics for sample

    # Calculate the weighted percentage difference
    percentage_diff = 0
    for key in weights.keys():
        percentage_diff += weights[key] * abs((sample_metrics[key] - target_metrics[key]) / target_metrics[key]) * 100

    return percentage_diff

In [11]:
# Simulated annealing function to find the best rows
def simulated_annealing_sampling(df_benchmark, df_target, weights, maxiter=500):
    target_metrics = calculate_metrics(df_target, len(df_target), weights)  # Get target metrics
    
    bounds = [(0, len(df_benchmark) - 1) for _ in range(len(df_target))]  # Define the range for row selection
    
    result = dual_annealing(objective_function, bounds, args=(df_benchmark, df_target, target_metrics, weights), maxiter=maxiter)
    
    selected_indices = [int(i) for i in result.x]  # Get the best rows selected
    return selected_indices


In [14]:
# Function to write results to Excel, updated to take both df_benchmark and df_target
def write_results_to_excel(benchmark_df, target_df, selected_indices, filename='benchmark_target.xlsx'):
    with pd.ExcelWriter(filename, mode='a', if_sheet_exists='replace') as writer:
        new_sample_col = f'Sample {len(benchmark_df.columns) - 1}'  # For a new sample column

        # Add a new column for selected rows
        benchmark_df[new_sample_col] = 0
        benchmark_df.loc[selected_indices, new_sample_col] = 1  # Set to 1 for selected rows

        # Write both DataFrames to separate sheets in the same Excel file
        benchmark_df.to_excel(writer, sheet_name='Benchmark', index=False)
        target_df.to_excel(writer, sheet_name='Target', index=False)

        writer.save()

In [15]:
# Main function to run the process
def main():
    df_benchmark, df_target = load_datasets('benchmark_target.xlsx')  # Load datasets from Excel

    # Define the column weights
    weights = {
        'Deb': 0.1,
        'GR': 0.2,
        'DPD': 0.2,
        'LP': 0.2,
        'collection': 0.1,
        'payments': 0.1
    }

    # Perform simulated annealing to find the best rows
    selected_indices = simulated_annealing_sampling(df_benchmark, df_target, weights)

    # Write the results back to the Excel file, adding a new sample column
    write_results_to_excel(df_benchmark, df_target, selected_indices)

# Run the main function
if __name__ == "__main__":
    main()

C:\Users\Gaming PC\AppData\Local\Temp\ipykernel_1820\238021454.py:14: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
